# Installs

In [1]:
import sys

# For conda, use the following syntax
# !conda install --yes --prefix {sys.prefix} library_name

# For pip, use the following syntax:
# !{sys.executable} -m pip install library_name
# !{sys.executable} -m pip install psycopg2-binary

In [ ]:
import xml.sax
import psycopg2
from psycopg2 import Error

class Article:
    def __init__(self, key, title="", journal="", year="", authors=[]):
        self.key = key
        self.title = title
        self.journal = journal
        self.year = year
        self.authors = authors
    
    # Setters        
    def set_title(self, title):
        self.title = title
        
    def set_journal(self, journal):
        self.journal = journal
        
    def set_year(self, year):
        self.year = year
        
    def set_authors(self, authors):
        self.authors = authors
        
    def add_author(self, author):
        self.authors.append(author)
    
    # Getters
    def get_key(self):
        return self.key
        
    def get_title(self):
        return self.title
        
    def get_journal(self):
        return self.journal
        
    def get_year(self):
        return self.year
    
    def get_authors(self):
        return self.authors
        
class Inproceedings:
    def __init__(self, key, title="", booktitle="", year="", authors=[]):
        self.key = key
        self.title = title
        self.booktitle = booktitle
        self.year = year
        self.authors = authors
    
    # Setters
    def set_title(self, title):
        self.title = title
        
    def set_booktitle(self, booktitle):
        self.booktitle = booktitle
        
    def set_year(self, year):
        self.year = year
        
    def set_authors(self, authors):
        self.authors = authors
        
    def add_author(self, author):
        self.authors.append(author)
        
    # Getters
    def get_key(self):
        return self.key
        
    def get_title(self):
        return self.title
        
    def get_booktitle(self):
        return self.booktitle
        
    def get_year(self):
        return self.year
    
    def get_authors(self):
        return self.authors
        

class TagHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.article = None
        self.inproceedings = None
        self.current_tag = None
        self.article_count = 0
        self.inproceedings_count = 0
        self.current_author = ''
        self.current_title = ''
        self.current_year = ''
        self.current_journal = ''
        self.current_booktitle = ''
        try:
            # Open Database Connection
            self.connection = psycopg2.connect(user="dblpuser",
                                                password="dblpus3r",
                                                host="localhost",
                                                port="5432",
                                                database="dblp")
            # Open a cursor to perform database operations
            self.cursor = self.connection.cursor()
            # Print PostgreSQL details
            print("PostgreSQL server information")
            print(self.connection.get_dsn_parameters(), "\n")
            # Executing a SQL query
            self.cursor.execute("SELECT version();")
            # Fetch result
            record = self.cursor.fetchone()
            print("You are connected to - ", record, "\n")
        except (Exception, Error) as error:
            print("Error while connecting to PostgreSQL", error)
            if self.connection is not None:
                self.cursor.close()
                self.connection.close()
                print("PostgreSQL connection is closed in __init__")           
        
    # Call when an element starts
    def startElement(self, tag, attributes):
        self.current_tag = tag
        if tag == "article":
            if self.inproceedings is not None:
                raise RuntimeException("Parse Error: start of article found before previous inproceedings end tag.")
            if self.article is not None:
                raise RuntimeException("Parse Error: start of article found before previous article end tag.")
            self.article = Article(attributes.getValue("key"), authors=[]) 
        elif tag == "inproceedings":
            if self.inproceedings is not None:
                raise RuntimeException("Parse Error: start of inproceedings found before previous inproceedings end tag.")
            if self.article is not None:
                raise RuntimeException("Parse Error: start of inproceedings found before previous article end tag.") 
            self.inproceedings = Inproceedings(attributes.getValue("key"), authors=[]) 
            
    # Call when an elements ends
    def endElement(self, tag):
        if tag == "article":            
            # Save article to database
            article_key = f"'{sanitizeString(self.article.get_key())}'"
            article_title = f"'{sanitizeString(self.article.get_title())}'" if self.article.get_title() != "" else 'NULL'
            article_journal = f"'{sanitizeString(self.article.get_journal())}'" if self.article.get_journal() != "" else 'NULL'
            article_year = f"'{sanitizeString(self.article.get_year())}'" if self.article.get_year() != "" else 'NULL'
            insert_query =  f"INSERT INTO public.\"Article\" (pubkey, title, journal, year) VALUES (" \
                            f"{article_key}, " \
                            f"{article_title}, " \
                            f"{article_journal}, " \
                            f"{article_year})"
            self.cursor.execute(insert_query)
            self.connection.commit() 
            # Save authors to database 
            for author in self.article.authors: # TODO: see if there is a way to read in unicode special characters
                insert_query =  f"INSERT INTO public.\"Authorship\" (pubkey, author) VALUES (" \
                            f"{article_key}, " \
                            f"'{sanitizeString(author)}')"
                self.cursor.execute(insert_query)
                self.connection.commit()
            self.article_count += 1
            self.article = None
        elif tag == "inproceedings": 
            # Save inproceedings to database     
            inproceedings_key = f"'{sanitizeString(self.inproceedings.get_key())}'"
            inproceedings_title = f"'{sanitizeString(self.inproceedings.get_title())}'" if self.inproceedings.get_title() != "" else 'NULL'
            inproceedings_booktitle = f"'{sanitizeString(self.inproceedings.get_booktitle())}'" if self.inproceedings.get_booktitle() != "" else 'NULL'
            inproceedings_year = f"'{sanitizeString(self.inproceedings.get_year())}'" if self.inproceedings.get_year() != "" else 'NULL'
            insert_query =  f"INSERT INTO public.\"Inproceedings\" (pubkey, title, booktitle, year) VALUES (" \
                            f"{inproceedings_key}, " \
                            f"{inproceedings_title}, " \
                            f"{inproceedings_booktitle}, " \
                            f"{inproceedings_year})"
            self.cursor.execute(insert_query)
            self.connection.commit()
            for author in self.inproceedings.authors:
                insert_query =  f"INSERT INTO public.\"Authorship\" (pubkey, author) VALUES (" \
                            f"{inproceedings_key}, " \
                            f"'{sanitizeString(author)}')"
                self.cursor.execute(insert_query)
                self.connection.commit()
            self.inproceedings_count += 1
            self.inproceedings = None
        elif self.current_tag == "author":
            if self.article is not None:
                # Check for duplicate authors
                if not self.current_author in self.article.get_authors():
                    self.article.add_author(self.current_author)
            elif self.inproceedings is not None:
                # Check for duplicate authors
                if not self.current_author in self.inproceedings.get_authors():
                    self.inproceedings.add_author(self.current_author) 
        elif self.current_tag == "title":
            if self.article is not None:
                self.article.set_title(self.current_title)
            elif self.inproceedings is not None:
                self.inproceedings.set_title(self.current_title)    
        elif self.current_tag == "year":
            if self.article is not None:
                self.article.set_year(self.current_year)
            elif self.inproceedings is not None:
                self.inproceedings.set_year(self.current_year)
        elif self.current_tag == "journal":
            if self.article is not None:
                self.article.set_journal(self.current_journal)
        elif self.current_tag == "booktitle":
            if self.inproceedings is not None:
                self.inproceedings.set_booktitle(self.current_booktitle)    
        self.current_tag = None
        self.current_author = ''
        self.current_title = ''
        self.current_year = ''
        self.current_journal = ''
        self.current_booktitle = ''
    
    # Call when document ends
    def endDocument(self):
        print(f"(end document) article count: {self.article_count}\ninproceedings count: {self.inproceedings_count}")
        if self.connection is not None:
            self.cursor.close()
            self.connection.close()
            print("PostgreSQL connection is closed in endDocument")

    # Call when a character is read
    def characters(self, content):
        if self.current_tag == "author":
            self.current_author += content
        elif self.current_tag == "title":
            self.current_title += content   
        elif self.current_tag == "year":
            self.current_year += content
        elif self.current_journal == "journal":
            self.current_author += content
        elif self.current_tag == "booktitle":
            self.current_booktitle += content
                
    def closeConnection(self):
        if self.connection is not None:
            self.cursor.close()
            self.connection.close()
            print("PostgreSQL connection is closed in closeConnection")
        

def sanitizeString(string):
    return string.replace("'","''").replace('"','""')
    
    
if( __name__ == "__main__"):
    # create an XMLReader
    parser = xml.sax.make_parser()
    # turn off namepsaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    # override the default ContextHandler
    Handler = TagHandler()
    parser.setContentHandler( Handler )
    parser.parse("dblp-2022-01-01.xml") # parser.parse("dblp-test.xml") 

PostgreSQL server information
{'user': 'dblpuser', 'channel_binding': 'prefer', 'dbname': 'dblp', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.1 on x86_64-apple-darwin20.6.0, compiled by Apple clang version 12.0.0 (clang-1200.0.32.29), 64-bit',) 

